In [1]:
import pandas as pd
import requests
import json
from config import api_key1, api_key2, api_key3, api_key4, api_key5, api_key6
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


In [2]:
#Load CSV Files with movie data
imdb_file = pd.read_csv("Resource/IMDb_All_Genres_etf_clean1.csv")
imdb_clean = imdb_file.drop(columns = ["Censor", "main_genre", "side_genre", "Runtime(Mins)"])
imdb_clean = imdb_clean.rename(columns = {"Rating": "imdb_score", "Total_Gross" : "Gross"})
imdb_clean.set_index("Movie_Title")
imdb_clean["Gross"] = imdb_clean["Gross"].replace({"Gross Unkown": ""})
imdb_clean["Gross"] = imdb_clean["Gross"].replace('[\$,]', '', regex=True)
imdb_clean["Gross"] = imdb_clean["Gross"].replace('[M,]', '', regex=True)
imdb_clean["Gross"] = pd.to_numeric(imdb_clean["Gross"])
#imdb_clean["Director"] = imdb_clean["Director"].replace('[\Directors:,]', '', regex=True)
imdb_clean.head()

,Movie_Title,Year,Director,Actors,imdb_score,Gross
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,NaN
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,534.86
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,377.85
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,292.58
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,342.55


In [3]:
#Load CSV Files with movie data
top1000_file = pd.read_csv("Resource/Top_1000_Highest_Grossing_Movies_Of_All_Time.csv")
top1000_file = top1000_file.drop(columns = ["Genre", "Movie Rating", "Duration", "Votes", "Logline"])
top1000_file = top1000_file.rename(columns = {"Movie Title": "Movie_Title", "Year of Realease": "Year"})
top1000_file = top1000_file.set_index("Movie_Title")
top1000_file["Year"] = top1000_file["Year"].replace({"X2016": "2016"})
top1000_file["Metascore"] = top1000_file["Metascore"].replace({"******": ""})
top1000_file["Year"] = pd.to_numeric(top1000_file["Year"])
top1000_file["Metascore"] = pd.to_numeric(top1000_file["Metascore"])
top1000_file["Gross"] = top1000_file["Gross"].replace('[\$,]', '', regex=True)
top1000_file["Gross"] = top1000_file["Gross"].replace('[M,]', '', regex=True)
top1000_file["Gross"] = top1000_file["Gross"].replace({"******": ""})
top1000_file["Gross"] = pd.to_numeric(top1000_file["Gross"])
top1000_file["Worldwide LT Gross"] = top1000_file["Worldwide LT Gross"].replace('[\$,]', '', regex=True)
top1000_file.dtypes

Year                    int64
Gross                 float64
Worldwide LT Gross     object
Metascore             float64
dtype: object

In [4]:
#Merge Dataframes into one
merge_df = pd.merge(imdb_clean, top1000_file, how = "outer", on = ["Movie_Title", "Year", "Gross"])
merge_df = merge_df.drop(columns = "Worldwide LT Gross")
merge_df.head()

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,NaN,NaN
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,534.86,84.0
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,377.85,94.0
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,292.58,74.0
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,342.55,87.0


In [5]:
# Set up additional columns
merge_df["Released"] = ""
merge_df["Rotten Tomatoes"] = ""
merge_df = merge_df.dropna(subset = ["Director"])
merge_df.count()

Movie_Title        5562
Year               5562
Director           5562
Actors             5562
imdb_score         5562
Gross              4701
Metascore           909
Released           5562
Rotten Tomatoes    5562
dtype: int64

In [16]:
requests.get("https://www.omdbapi.com/?t=Allegiant&apikey=ee160a48").json()

{'Title': 'Allegiant',
 'Year': '2016',
 'Rated': 'PG-13',
 'Released': '18 Mar 2016',
 'Runtime': '120 min',
 'Genre': 'Action, Adventure, Mystery',
 'Director': 'Robert Schwentke',
 'Writer': 'Noah Oppenheim, Adam Cooper, Bill Collage',
 'Actors': 'Shailene Woodley, Theo James, Jeff Daniels',
 'Plot': 'After the earth-shattering revelations of Insurgent, Tris must escape with Four beyond the wall that encircles Chicago, to finally discover the shocking truth of the world around them.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '3 wins & 12 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNjk2OTc5YzQtMjAwMS00YWY4LTk1ZWItOTgyMmRkMGU4ZmY1XkEyXkFqcGdeQXVyMzQ1MzUwMTE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.7/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '11%'},
  {'Source': 'Metacritic', 'Value': '33/100'}],
 'Metascore': '33',
 'imdbRating': '5.7',
 'imdbVotes': '131,911',
 'imdbID': 'tt3410834',
 'Type': 'mo

In [29]:
#Iterate through merged_df Dataframe
import time
counter = 0
baseurl = "https://www.omdbapi.com/?t="
print("Accessing API")

for index, row in merge_df.iterrows():
    counter += 1
    if counter < 950:
        apikey = api_key6
    elif counter < 1900:
        apikey = api_key2
    elif counter < 2850:
        apikey = api_key3
    elif counter < 3800:
        apikey = api_key4
    elif counter < 4750:
        apikey = api_key5
    elif counter < 5562:
        apikey = api_key1
    
    #Get movie title from merge_df
    movie_title = merge_df.loc[index, "Movie_Title"]
    
    #API Request
    time.sleep(0.01)
    filminfo = requests.get(f"{baseurl}{movie_title}&apikey={apikey}").json()
    
    #Store Release Date, Rotten Tomatoes, Metascore,Box Office
    try: 
        merge_df.loc[index, "Released"] = filminfo["Released"]
        merge_df.loc[index, "Rotten Tomatoes"] = filminfo["Ratings"][1]["Value"]
        merge_df.loc[index, "Metascore"] = filminfo["Metascore"]
        merge_df.loc[index, "Gross"] = filminfo["BoxOffice"]
    except:
        pass
    
#Display Data
merge_df

Accessing API


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [31]:
merge_df.head(500)

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore,Released,Rotten Tomatoes
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,N/A,N/A,30 Sep 2022,80%
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,"$534,987,076",84,18 Jul 2008,94%
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,"$379,427,292",94,17 Dec 2003,94%
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,"$292,587,330",74,16 Jul 2010,87%
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,"$342,952,511",87,18 Dec 2002,95%
...,...,...,...,...,...,...,...,...,...
495,The Unbearable Weight of Massive Talent,2022,Tom Gormican,"Nicolas Cage, Pedro Pascal, Tiffany Haddish, S...",7.0,"$20,300,157",68,22 Apr 2022,87%
496,Cloverfield,2008,Matt Reeves,"Mike Vogel, Jessica Lucas, Lizzy Caplan, T.J. ...",7.0,"$80,048,433",64,18 Jan 2008,78%
497,Death Proof,2007,Quentin Tarantino,"Kurt Russell, Zoë Bell, Rosario Dawson, Vaness...",7.0,N/A,N/A,31 May 2007,67%
498,Tropic Thunder,2008,Ben Stiller,"Ben Stiller, Jack Black, Robert Downey Jr., Je...",7.0,"$110,515,313",71,13 Aug 2008,82%


In [ ]:
directorframe = merge_df["Director"].str.split(", ", expand = True, n = 1)
directorframe= directorframe.rename(columns = {directorframe.columns[0]: "Director"})
merge_df["Director"] = directorframe["Director"]
merge_df.head()

In [ ]:
actorframe = merge_df["Actors"].str.split(", ", expand = True, n = 1)
actorframe= actorframe.rename(columns = {actorframe.columns[0]: "Actors"})
merge_df["Actors"] = actorframe["Actors"]
merge_df.head()

In [ ]:
merge_df["Gross"] = merge_df["Gross"].replace('[\$,]', '', regex=True)
merge_df["Rotten Tomatoes"] = merge_df["Rotten Tomatoes"].replace('[%]', '', regex=True)
merge_df["Released"] = merge_df["Released"].replace('[0123456789 ]', '', regex=True)
#Converting IMDB to out of 100 score instead of out of 10
merge_df["imdb_score"] = merge_df["imdb_score"] * 10
merge_df

In [ ]:
merge_df["Gross"] = pd.to_numeric(merge_df["Gross"])
merge_df["Metascore"] = pd.to_numeric(merge_df["Metascore"])
merge_df["Rotten Tomatoes"] = pd.to_numeric(merge_df["Rotten Tomatoes"])
merge_df.dtypes

In [ ]:
x_values = merge_df["imdb_score"]
y_values = merge_df["Gross"]
plt.scatter(x_values, y_values)
#plt.ylim(0, 850000000)
#plt.xlim(0,100)
plt.ylabel("Gross Income in Million$")
plt.xlabel("IMDB Score (*10)")
plt.show()
imdb_reg = linregress(x_values, y_values)
imdb_line = imdb_reg.slope*x_values+imdb_reg.intercept
imdb_reg.slope
#plt.plot(x_values, imdb_line, "r")

In [ ]:
x_values = merge_df["Metascore"]
y_values = merge_df["Gross"]
plt.scatter(x_values, y_values)
#plt.ylim(0, 850000000)
#plt.xlim(0,100)
plt.ylabel("Gross Income in Million$")
plt.xlabel("Metascore")
plt.show()
meta_reg = linregress(x_values, y_values)
meta_line = meta_reg.slope*x_values+meta_reg.intercept

In [ ]:
x_values = merge_df["Rotten Tomatoes"]
y_values = merge_df["Gross"]
plt.scatter(x_values, y_values)
#plt.ylim(0, 850000000)
#plt.xlim(0,100)
plt.ylabel("Gross Income in Million$")
plt.xlabel("Rotten Tomatoes (%)")
plt.show()
rot_reg = linregress(x_values, y_values)
rot_line = rot_reg.slope*x_values+rot_reg.intercept

In [ ]:
#Find per film average in revenue by director
directors = merge_df[["Director", "Gross"]]
director_average = directors.groupby(["Director"]).mean().sort_values(["Gross"], ascending = False)
director_average.head(50)